# LangGraph: Agent Executor

[LangGraph](https://python.langchain.com/docs/langgraph) \
[Tutorial on Youtube](https://www.youtube.com/watch?v=9dXp5q3OFdQ&list=PLfaIDFEXuae16n2TWUkKq5PgJ0w6Pkwtg&index=3&ab_channel=LangChain) \
[Tools: Google Search](https://python.langchain.com/docs/integrations/tools/google_search)

In [1]:
import os
import sys

from dotenv import load_dotenv

sys.path.append(os.path.abspath(os.pardir))
load_dotenv('../.env')

False

In [2]:
assert os.environ['LANGCHAIN_TRACING_V2']

In [3]:
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name='gemini-pro')

In [4]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper(k=5)


def top5_results(query) -> str:
    return search.results(query, 5)


tools = [
    Tool(
        name="Google Search Snippets",
        description="Search Google for recent results.",
        func=search.run,
        # func=top5_results,
    )
]

In [5]:
from langchain import hub

prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [6]:
from langchain.agents import create_react_agent

agent_runnable = create_react_agent(llm, tools, prompt)

In [7]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [8]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)


def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}


def execute_tools(data):
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}


def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"

In [9]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)

workflow.add_edge("action", "agent")

app = workflow.compile()

In [10]:
inputs = {"input": "what is the weather in sf", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentAction(tool='Google Search Snippets', tool_input='what is the weather in sf', log='Action: Google Search Snippets\nAction Input: what is the weather in sf')}
----
{'intermediate_steps': [(AgentAction(tool='Google Search Snippets', tool_input='what is the weather in sf', log='Action: Google Search Snippets\nAction Input: what is the weather in sf'), "Be prepared with the most accurate 10-day forecast for San Francisco, CA with highs, lows, chance of precipitation from The Weather Channel and Weather.com. Hourly Weather · 1 AM 51°. rain drop 59% · 2 AM 51°. rain drop 49% · 3 AM 51°. rain drop 49% · 4 AM 51°. rain drop 69% · 5 AM 51°. rain drop 49% · 6 AM 51°. Today's and tonight's San Francisco, CA weather forecast, weather conditions and Doppler radar from The Weather Channel and Weather.com. Current conditions at. SAN FRANCISCO DOWNTOWN (SFOC1). Lat: 37.77056°NLon: 122.42694°WElev: 150.0ft. NA. 53°F. 12°C. Humidity, 79%. Wind Speed, NA NA MPH. 8:00 pm · Feels Lik

In [11]:
output = app.invoke(inputs)
output

{'input': 'what is the weather in sf',
 'chat_history': [],
 'agent_outcome': AgentFinish(return_values={'output': 'There is rain in SF with 59% chance of precipitation and temperature of 51° F'}, log=' I now know the final answer\nFinal Answer: There is rain in SF with 59% chance of precipitation and temperature of 51° F'),
 'intermediate_steps': [(AgentAction(tool='Google Search Snippets', tool_input='what is the weather in sf', log='Action: Google Search Snippets\nAction Input: what is the weather in sf'),
   "Be prepared with the most accurate 10-day forecast for San Francisco, CA with highs, lows, chance of precipitation from The Weather Channel and Weather.com. Hourly Weather · 1 AM 51°. rain drop 59% · 2 AM 51°. rain drop 49% · 3 AM 51°. rain drop 49% · 4 AM 51°. rain drop 69% · 5 AM 51°. rain drop 49% · 6 AM 51°. Today's and tonight's San Francisco, CA weather forecast, weather conditions and Doppler radar from The Weather Channel and Weather.com. Current conditions at. SAN FRA

In [12]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content=inputs['input']),
    AIMessage(content=output['agent_outcome'].return_values['output'])
]
inputs = {"input": "then what is the weather in LA", "chat_history": chat_history}

In [13]:
output = app.invoke(inputs)
output

{'input': 'then what is the weather in LA',
 'chat_history': [HumanMessage(content='what is the weather in sf'),
  AIMessage(content='There is rain in SF with 59% chance of precipitation and temperature of 51° F')],
 'agent_outcome': AgentFinish(return_values={'output': 'According to the weather channel, the current weather in Los Angeles, CA is 55 degrees Fahrenheit and feels like 55 degrees. The humidity is 80%, there is 0% chance of rain, and the wind is 2 mph.'}, log='Final Answer: According to the weather channel, the current weather in Los Angeles, CA is 55 degrees Fahrenheit and feels like 55 degrees. The humidity is 80%, there is 0% chance of rain, and the wind is 2 mph.'),
 'intermediate_steps': [(AgentAction(tool='Google Search Snippets', tool_input='la weather', log='Action: Google Search Snippets\nAction Input: la weather'),
   "Be prepared with the most accurate 10-day forecast for Los Angeles, CA with highs, lows, chance of precipitation from The Weather Channel and Weath

In [14]:
output['agent_outcome'].return_values['output']

'According to the weather channel, the current weather in Los Angeles, CA is 55 degrees Fahrenheit and feels like 55 degrees. The humidity is 80%, there is 0% chance of rain, and the wind is 2 mph.'